In [ ]:
!pip install git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.keras import EfficientNetB7

In [ ]:
from efficientnet.keras import EfficientNetB3

In [ ]:
import tensorflow as tf
import zipfile
import pandas as pd
import keras
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
train_path = '../input/shopee-code-league-2020-product-detection/resized/train/'
test_path = '../input/shopee-code-league-2020-product-detection/resized/test/'

In [ ]:
df = pd.read_csv("../input/shopee-code-league-2020-product-detection/train.csv")
df = df.sample(frac=1)
df.head()

In [ ]:
test_df = pd.read_csv("../input/shopee-code-league-2020-product-detection/test.csv")
test_df.head()

In [ ]:
df['category'] = df['category'].astype(int)
df['category'] = df['category'].apply(lambda x: "{:02d}".format(x))
cats = df['category']

In [ ]:
df['combined_filename'] = df['category'].map(lambda x: x + "/").astype(str) + df['filename']
df.head()

In [ ]:
mod_df = df.drop(columns = ["filename"])
mod_df.head()

In [ ]:
mod_df = mod_df.sample(frac=1)
mod_df.head()

In [ ]:
for cat in cats:
  path = os.path.join(train_path, cat)
  for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path, img))
    plt.imshow(img_array, cmap='gray')
    plt.show()
    break
  break

In [ ]:
new_array = cv2.resize(img_array, (256,256))
plt.imshow(new_array)
plt.show()

In [ ]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.6)
IMG_SIZE = 300

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="validation",
batch_size=32,
seed=40,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory=test_path,
x_col="filename",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
# Create the base model from the pre-trained model EfficientNetB7
base_model = EfficientNetB7(input_shape=IMG_SHAPE,
                           include_top=False,
                            weights='imagenet')

In [ ]:
# Freeze the base
base_model.trainable = False

In [ ]:
dropout_layer = tf.keras.layers.Dropout(0.3)

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer = keras.layers.Dense(42)

In [ ]:
from keras.optimizers import Adam
new_model = tf.keras.Sequential([
  base_model,
  dropout_layer,
  global_average_layer,
  prediction_layer
])

In [ ]:
new_model.compile(Adam(lr=3e-4),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size + 1
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size + 1
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size + 1
new_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4
)

In [ ]:
new_model.save("efficientnet_4epochs_0.03valid")

In [ ]:
ls

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 400

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(Adam(lr=5e-5),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
new_model.summary()

In [ ]:
mod_df = mod_df.sample(frac=1)

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=mod_df,
directory=train_path,
x_col="combined_filename",
y_col="category",
subset="validation",
batch_size=32,
seed=40,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
new_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3
)

In [ ]:
test_generator.reset()
pred=new_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
res = test_df.drop(columns='category')
res['category'] = pd.DataFrame(predictions).astype(int)
res['category'] = res['category'].apply(lambda x: "{:02d}".format(x))

In [ ]:
final = test_df.drop(columns='category')
final['category'] = pd.DataFrame(predictions).astype(int)

In [ ]:
final['category'] = final['category'].apply(lambda x: "{:02d}".format(x))

In [ ]:
final.head()

In [ ]:
final.to_csv("submissions.csv", index=False)

In [ ]:
ls

In [ ]:
final_model = new_model

In [ ]:
final_model.save("256x256EfficientNetB7")

In [ ]:
final_model.save_weights("256x256_8epochs_0.15valid_weights")

In [ ]:
ls

In [ ]:
corr_df = df
corr_df = corr_df.sample(frac=1)
corr_df

In [ ]:
corr_datagen=ImageDataGenerator(rescale=1./255.)
corr_generator=corr_datagen.flow_from_dataframe(
dataframe=corr_df,
directory=train_path,
x_col="combined_filename",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(IMG_SIZE,IMG_SIZE))

In [ ]:
STEP_SIZE_CORR = corr_generator.n//corr_generator.batch_size + 1
corr_generator.reset()
corr_pred=new_model.predict_generator(corr_generator,
steps=STEP_SIZE_CORR,
verbose=1)

In [ ]:
corr_predicted_class_indices=np.argmax(corr_pred,axis=1)

In [ ]:
corr_labels = (train_generator.class_indices)
corr_labels = dict((v,k) for k,v in corr_labels.items())
corr_predictions = [labels[k] for k in corr_predicted_class_indices]

In [ ]:
print(len(corr_predictions))

In [ ]:
df['category'].value_counts()

In [ ]:
corr_df['category'] = pd.DataFrame(corr_predictions).astype(int)
corr_df['category'].value_counts()

In [ ]:
corr_df.drop(columns=['category'])
corr_df['category'] = pd.DataFrame(corr_predictions).astype(int)
corr_df

In [ ]:

corr_df.to_csv("corr.csv", index=False)

In [ ]:
ls

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size + 1
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size + 1
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size + 1
new_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)